In [1]:
import pandas as pd
import requests

In [2]:
df = pd.read_csv('distritos_ciudade_lat_long.csv', header=None)
df.columns = ['distrito', 'lat', 'long']
df.head(20)

,distrito,lat,long
0,San Borja,-12.088989,-76.983337
1,Lurin,-12.285038,-76.842918
2,San Bartolo,-12.394174,-76.751937
3,Ventanilla,-11.865615,-77.073974
4,Tumbes,-3.858614,-80.580500
5,Jaen,-5.811504,-78.822688
6,Pucallpa,-8.355925,-74.647883
7,Lima,-12.043181,-77.028236
8,Arequipa,-16.398889,-71.535004
9,El Callao,-12.056590,-77.118141


In [3]:
df.dtypes

distrito     object
lat         float64
long        float64
dtype: object

In [8]:
dd = pd.read_csv('repo_tiendas_peru_cima.csv')
dd.head()

,id,title,lng,address,phone,country_code,code,type,delivery,store_url,promo1,promo2,promo3,data
0,1356743,Tu Bodega,-77.036748,Av. Victor Larco 0466,962303360.0,51,10065187,Particular,1,NaN,NaN,NaN,NaN,NaN
1,1357521,bodega bar biki,-76.383563,Urb. Los Cipreses Mzn D Lote 03,952698941.0,51,10073110,Bodega,0,NaN,NaN,NaN,NaN,eyJwcm9tbzEiOiIifQ==
2,1357522,Tu Bodega,-76.371006,Av. Mariscal Benavides 1355,999551871.0,51,10073121,Bodega,0,NaN,NaN,NaN,NaN,NaN
3,1357523,Tu Bodega,-76.386392,Av. 28 De Julio 263,991530646.0,51,10073125,Bodega,0,NaN,NaN,NaN,NaN,NaN
4,1357524,Tu Bodega,-76.356483,S/V Sindicato De Choferes Mzn Q Lot,991901405.0,51,10073133,Bodega,0,NaN,NaN,NaN,NaN,NaN


In [9]:
dd.dtypes

id                int64
title            object
lng             float64
address          object
phone           float64
country_code      int64
code              int64
type             object
delivery          int64
store_url       float64
promo1          float64
promo2          float64
promo3          float64
data             object
dtype: object

In [10]:
id_tiendas_existentes = list(dd['id'].unique())
#id_tiendas_existentes = []
len(id_tiendas_existentes)

87415

In [7]:
import requests
import json

url = "https://1fzqk3npw4.execute-api.us-east-1.amazonaws.com/nearby_store_stage/pe"

headers = {
    'content-type': "application/json",
    'cache-control': "no-cache"
    }


def get_data_tiendas(lat, long):
    payload = {
        "south":lat - 2.5,
        "west":long - 2.5,
        "north":lat + 2.5,
        "east":long + 2.5,
        "center":{"lat":lat,"lng":long},
        "zoom":750,
        "country_code":"pe"
    }
    
    response = requests.request("POST", url, data=json.dumps(payload), headers=headers)
    result = response.json()
    
    return result

## Recorriendo las latitudes a considerar

In [8]:
for row in df.iterrows():
    print(row[1]['distrito'], type(row[1]['lat']), type(row[1]['long']))
    
    result = get_data_tiendas(row[1]['lat'], row[1]['long'])
    
    _items = result.items()
    dicc = {
        'id': [v.get('id', 0) for k, v in _items],
        'title': [v.get('title', '') for k, v in _items],
        'lng': [v.get('lat', 0.0) for k, v in _items],
        'lng': [v.get('lng', 0.0) for k, v in _items],
        'address': [v.get('address', '') for k, v in _items],
        'phone': [str(v.get('phone', '')) for k, v in _items],
        'country_code': [v.get('country_code', '') for k, v in _items],
        'code': [v.get('code', '') for k, v in _items],
        'type': [v.get('type', '') for k, v in _items],
        'delivery': [v.get('delivery', 0) for k, v in _items],
        'store_url': [v.get('store_url', '') for k, v in _items],
        'promo1': [v.get('promo1', '') for k, v in _items],
        'promo2': [v.get('promo2', 0) for k, v in _items],
        'promo3': [v.get('promo3', 0) for k, v in _items],
        'data': [v.get('data', 0) for k, v in _items]
    }
    cercanos = pd.DataFrame(dicc)
    print("detectados: ", cercanos.shape[0])
    
    cercanos = cercanos[~cercanos['id'].isin(id_tiendas_existentes)]
    _len_nuevos = cercanos.shape[0]
    print("nuevos: ", cercanos.shape[0])
    
    # Agregado de nuevos por zonas
    if _len_nuevos > 0:
        dd = pd.concat([dd, cercanos], ignore_index=True, axis=0)
        id_tiendas_existentes = list(dd['id'].unique())
        print(len(id_tiendas_existentes))
        #dd.to_csv("repo_tiendas_peru_cima.csv", index=False)
    
    print("/"*50)

San Borja <class 'float'> <class 'float'>
detectados:  18750
nuevos:  3773
81656
//////////////////////////////////////////////////
Lurin <class 'float'> <class 'float'>
detectados:  18750
nuevos:  4
81660
//////////////////////////////////////////////////
San Bartolo <class 'float'> <class 'float'>
detectados:  18750
nuevos:  0
//////////////////////////////////////////////////
Ventanilla <class 'float'> <class 'float'>
detectados:  18750
nuevos:  0
//////////////////////////////////////////////////
Tumbes <class 'float'> <class 'float'>
detectados:  10811
nuevos:  0
//////////////////////////////////////////////////
Jaen <class 'float'> <class 'float'>
detectados:  18750
nuevos:  481
82141
//////////////////////////////////////////////////
Pucallpa <class 'float'> <class 'float'>
detectados:  5197
nuevos:  0
//////////////////////////////////////////////////
Lima <class 'float'> <class 'float'>
detectados:  18750
nuevos:  0
//////////////////////////////////////////////////
Arequipa 

In [9]:
#lat = -12.24896638582862
#long = -76.90411419839475

#lat - 1, lat + 1, long -1, long + 1

In [10]:
dd.shape, dd['id'].unique().shape

((240985, 14), (87415,))

In [11]:
dd.drop_duplicates(
    subset=['id'], keep='first', inplace=True, ignore_index=True
)

In [12]:
dd.shape, dd['id'].unique().shape

((87415, 14), (87415,))

In [13]:
dd['country_code'].value_counts()

51     77883
+51     9532
Name: country_code, dtype: int64

In [15]:
dd.to_csv("repo_tiendas_peru_cima.csv", index=False)

#repo_tiendas_peru_cima